In [1]:
import os
import sys
from pathlib import Path

import graphviz
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def load_npzs(data_dir: str):
    return [
        np.load(data_dir + "/" + str(f))
        for f in os.listdir(data_dir)
        if f.endswith(".npz")
    ]


def load_all_layouts(layout_dir: str):
    return {
        "ndt": load_npzs(layout_dir + "/nlp/default/train"),
        "nrt": load_npzs(layout_dir + "/nlp/random/train"),
        "xdt": load_npzs(layout_dir + "/xla/default/train"),
        "xrt": load_npzs(layout_dir + "/xla/random/train"),
    }

In [3]:
npzs = load_all_layouts(r"H:\data\pamr\predict-ai-model-runtime\npz_all\npz\layout")
npzs.keys()

dict_keys(['ndt', 'nrt', 'xdt', 'xrt'])

In [4]:
npzs["ndt"][0].files

['node_feat',
 'node_opcode',
 'edge_index',
 'node_config_feat',
 'node_config_ids',
 'config_runtime',
 'node_splits']

In [5]:
npzs["xdt"][0]["edge_index"].shape

(597, 2)

In [6]:
npzs["xdt"][0]["node_opcode"].shape

(372,)

## Draw model graph

In [7]:
import yaml

with open("../../data/node_idx2name.yaml") as f:
    node_idx2name = yaml.load(f, Loader=yaml.FullLoader)

In [8]:
model = npzs["xdt"][0]
model_name = model.fid.name.split("/")[-1].split(".")[0]

# Create a new directed graph
dot = graphviz.Digraph(model_name)

# Add nodes and edges to the graph
for edge in model["edge_index"]:
    dot.edge(str(edge[0]), str(edge[1]))
    
for node in model["node_config_ids"]:
    dot.node(str(node), fillcolor="red", style="filled")

for node, node_idx in enumerate(model["node_opcode"]):
    dot.node(str(node), label=node_idx2name[node_idx])

dot.render(f"../../output/{model_name}.gv", view=True)

'..\\..\\output\\alexnet_train_batch_32.gv.pdf'